In [16]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torchvision.models as models

import timm  # Thư viện timm cung cấp các mô hình transformer như DeiT

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Sử dụng thiết bị: {device}")

Sử dụng thiết bị: cuda


In [18]:
df = pd.read_csv("/kaggle/input/ai-1810-dpl-302-m-butterfly-image-classification/Training_set.csv")
df.head(10)

,filename,label
0,Image_1.jpg,SOUTHERN DOGFACE
1,Image_2.jpg,ADONIS
2,Image_3.jpg,BROWN SIPROETA
3,Image_4.jpg,MONARCH
4,Image_5.jpg,GREEN CELLED CATTLEHEART
5,Image_6.jpg,CAIRNS BIRDWING
6,Image_7.jpg,GREEN CELLED CATTLEHEART
7,Image_8.jpg,EASTERN DAPPLE WHITE
8,Image_9.jpg,BROWN SIPROETA
9,Image_10.jpg,RED POSTMAN


In [19]:
len(df)

5000

In [20]:
if 'label' in df.columns:
    num_classes = df['label'].nunique()
    label_column = 'label'
    print(f"Số lớp bướm khác nhau: {num_classes}")
else:
    possible_label_columns = ['category', 'class', 'type', 'species']
    for col in possible_label_columns:
        if col in df.columns:
            label_column = col
            num_classes = df[col].nunique()
            print(f"Tìm thấy cột nhãn: {label_column}")
            print(f"Số lớp bướm khác nhau: {num_classes}")
            break
    else:
        print("Không tìm thấy cột nhãn rõ ràng. Giả định cột thứ hai là nhãn.")
        label_column = df.columns[1]
        num_classes = df[label_column].nunique()
        print(f"Sử dụng cột {label_column} làm nhãn với {num_classes} lớp")

Số lớp bướm khác nhau: 75


In [21]:
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [22]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import warnings


In [23]:
image_dir = "/kaggle/input/ai-1810-dpl-302-m-butterfly-image-classification/train/train"

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df[label_column], random_state=42)
print(f"Số lượng ảnh huấn luyện: {len(train_df)}")
print(f"Số lượng ảnh validation: {len(val_df)}")

Số lượng ảnh huấn luyện: 4000
Số lượng ảnh validation: 1000


In [24]:
class ButterflyDataset(Dataset):
    def __init__(self, data_frame, img_dir, transform=None, is_test=False, label_col='label'):
        self.data_frame = data_frame
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        self.label_col = label_col
        
        if not is_test:
            self.classes = sorted(data_frame[label_col].unique())
            self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
    
    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        img_name = self.data_frame.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)
        if not os.path.exists(img_path):
            for ext in ['.jpg', '.jpeg', '.png']:
                if os.path.exists(img_path + ext):
                    img_path = img_path + ext
                    break
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Lỗi khi đọc ảnh {img_path}: {e}")
            image = Image.new('RGB', (IMAGE_SIZE, IMAGE_SIZE), color='gray')
        
        if self.transform:
            image = self.transform(image)
        
        if not self.is_test:
            label_name = self.data_frame.iloc[idx, 1] if self.label_col=='label' else self.data_frame.iloc[idx][self.label_col]
            label = self.class_to_idx[label_name]
            return image, label
        else:
            return image, img_name
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3))
])
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

try:
    train_dataset = ButterflyDataset(
        data_frame=train_df,
        img_dir=image_dir,
        transform=train_transforms,
        label_col=label_column
    )
    val_dataset = ButterflyDataset(
        data_frame=val_df,
        img_dir=image_dir,
        transform=val_transforms,
        label_col=label_column
    )
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
    
    print("Dataset và DataLoader đã được tạo thành công!")
except Exception as e:
    print(f"Lỗi khi tạo dataset: {e}")
    raise


Dataset và DataLoader đã được tạo thành công!


In [25]:
from torchvision import models

# Hàm tạo mô hình sử dụng AlexNet
def create_model(num_classes):
    model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)  # Dùng pretrained weights
    
    # Freeze toàn bộ các layers ngoại trừ classifier
    for param in model.features.parameters():
        param.requires_grad = False
    
    # Thay đổi fully connected cuối cùng
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(9216, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 1024),
        nn.ReLU(inplace=True),
        nn.Linear(1024, num_classes)
    )
    
    return model

# Khởi tạo mô hình
model = create_model(num_classes)
model = model.to(device)

In [26]:
EPOCH = 20

In [27]:
# Định nghĩa hàm mất mát với label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Sử dụng optimizer Adam với weight decay
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Scheduler Cosine Annealing
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCH)

In [31]:
OUTPUT_PATH = '/kaggle/working/'
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

In [32]:
import torch
import os
from tqdm import tqdm
def train_model(model, train_loader, val_loader, epoch, criterion, optimizer, scheduler=None):
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []    
    best_val_loss = float('inf')
    
    for epochs in range(epoch):
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0
        
        for images, labels in tqdm(train_loader, desc=f'Epoch {epochs+1}/{epoch} [Train]'):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            train_correct += (outputs.argmax(1) == labels).sum().item()
            train_total += labels.size(0)
        
        history['train_losses'].append(train_loss / len(train_loader.dataset))
        history['train_accs'].append(train_correct / train_total)
        
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f'Epoch {epochs+1}/{epoch} [Val]'):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * images.size(0)
                val_correct += (outputs.argmax(1) == labels).sum().item()
                val_total += labels.size(0)
        epoch_val_loss = val_loss / len(val_loader.dataset)
        history['val_losses'].append(val_loss / len(val_loader.dataset))
        history['val_accs'].append(val_correct / val_total)

        print(f'\nEpoch {epochs+1}/{epoch}: Train Loss: {history["train_losses"][-1]:.4f}, Train Acc: {history["train_accs"][-1]:.4f}, Val Loss: {history["val_losses"][-1]:.4f}, Val Acc: {history["val_accs"][-1]:.4f}')
        if scheduler:
            scheduler.step()
            
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            torch.save(model.state_dict(), os.path.join(OUTPUT_PATH, 'alexnet_model.pth'))

    return history


In [33]:
history = {'train_losses': [], 'train_accs': [], 'val_losses': [], 'val_accs': []}


In [34]:
history = train_model(model, train_loader, val_loader, 20, criterion, optimizer, scheduler)

Epoch 1/20 [Val]: 100%|██████████| 16/16 [00:01<00:00,  9.92it/s]



Epoch 1/20: Train Loss: 2.3375, Train Acc: 0.5200, Val Loss: 1.7419, Val Acc: 0.7250


Epoch 2/20 [Val]: 100%|██████████| 16/16 [00:01<00:00,  9.18it/s]



Epoch 2/20: Train Loss: 2.0170, Train Acc: 0.6192, Val Loss: 1.5917, Val Acc: 0.7670


Epoch 3/20 [Val]: 100%|██████████| 16/16 [00:01<00:00, 10.01it/s]


...


Epoch 18/20: Train Loss: 1.3191, Train Acc: 0.8630, Val Loss: 1.2488, Val Acc: 0.8870


Epoch 19/20 [Val]: 100%|██████████| 16/16 [00:01<00:00, 10.18it/s]



Epoch 19/20: Train Loss: 1.3005, Train Acc: 0.8695, Val Loss: 1.2480, Val Acc: 0.8880


Epoch 20/20 [Val]: 100%|██████████| 16/16 [00:01<00:00,  9.78it/s]


Epoch 20/20: Train Loss: 1.3209, Train Acc: 0.8562, Val Loss: 1.2480, Val Acc: 0.8880


In [41]:
Test_dir = "/kaggle/input/ai-1810-dpl-302-m-butterfly-image-classification/test/test"
def process_test_folder(test_dir, transform):
    if not os.path.exists(test_dir):
        print(f"Không tìm thấy thư mục test: {test_dir}")
        return None
    
    # Quét tất cả thư mục con và thu thập file ảnh hợp lệ
    test_files = []
    val_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
    
    for root, dirs, files in os.walk(test_dir):
        for file in files:
            if file.lower().endswith(val_extensions):
                # Lấy đường dẫn tương đối so với test_dir
                rel_path = os.path.relpath(os.path.join(root, file), test_dir)
                test_files.append(rel_path)
    
    if not test_files:
        print("Không tìm thấy file ảnh hợp lệ trong thư mục test!")
        return None
    
    print(f"Đã tìm thấy {len(test_files)} ảnh trong thư mục test và các thư mục con.")
    test_df = pd.DataFrame({'image': test_files})

    class TestDataset(Dataset):
        def __init__(self, file_list, dir_path, transform=None):
            self.file_list = file_list
            self.dir_path = dir_path
            self.transform = transform
        
        def __len__(self):
            return len(self.file_list)
        
        def __getitem__(self, idx):
            img_name = self.file_list[idx]
            img_path = os.path.join(self.dir_path, img_name)
            
            # Kiểm tra và xử lý lỗi đường dẫn
            if not os.path.isfile(img_path):
                print(f"Lỗi đường dẫn: {img_path} không phải là file!")
                return torch.zeros((3, 224, 224)), "invalid"
            
            try:
                image = Image.open(img_path).convert('RGB')
                if self.transform:
                    image = self.transform(image)
                return image, img_name
            except Exception as e:
                print(f"Lỗi khi xử lý ảnh {img_path}: {str(e)}")
                return torch.zeros((3, 224, 224)), "invalid"

    test_dataset = TestDataset(test_df['image'].values, test_dir, transform)
    test_loader = DataLoader(
        test_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=2,
        collate_fn=lambda x: (torch.stack([item[0] for item in x]), [item[1] for item in x])
    )
    
    return test_loader

# CẬP NHẬT HÀM DỰ ĐOÁN
def predict_and_submit(model, test_loader, class_mapping, output_path):
    model.eval()
    predictions = []
    filenames = []
    idx_to_class = {v: k for k, v in class_mapping.items()}
    
    with torch.no_grad():
        for images, batch_names in tqdm(test_loader, desc="Đang dự đoán..."):
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            
            predictions.extend([idx_to_class[idx.item()] for idx in predicted])
            filenames.extend(batch_names)
    
    # Lọc các dự đoán không hợp lệ
    val_data = [(f, p) for f, p in zip(filenames, predictions) if f != "invalid"]
    
    submission_df = pd.DataFrame({
        'Id': [f for f, p in val_data],
        'Category': [p for f, p in val_data]
    })
    
    submission_path = os.path.join(output_path, 'submission.csv')
    submission_df.to_csv(submission_path, index=False)
    print(f"Đã tạo file submission với {len(submission_df)} dự đoán hợp lệ.")
if os.path.exists(Test_dir):
    best_model = create_model(num_classes)
    best_model.load_state_dict(torch.load(os.path.join(OUTPUT_PATH, 'alexnet_model.pth')))
    best_model = best_model.to(device)
    
    test_loader = process_test_folder(Test_dir, val_transforms)
    
    if test_loader:
        predict_and_submit(best_model, test_loader, train_dataset.class_to_idx, OUTPUT_PATH)
    else:
        print("Không thể tạo test loader do thiếu dữ liệu.")


<ipython-input-41-1734976b1700>:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(os.path.join(OUTPUT_PATH, 'alexnet_model.pth')))


Đã tìm thấy 1499 ảnh trong thư mục test và các thư mục con.


Đang dự đoán...: 100%|██████████| 24/24 [00:03<00:00,  7.35it/s]

Đã tạo file submission với 1499 dự đoán hợp lệ.
